In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

In [62]:
stai_scores = np.array(pd.read_csv(r'stai_scores.csv', header=None)[0])
inst_choices = np.array(pd.read_csv(r'inst_choices.csv', header=None))
inst_outcomes = np.array(pd.read_csv(r'inst_outcomes.csv', header=None))
print(inst_outcomes)

[[0 0 1 ... 0 0 1]
 [0 1 1 ... 1 0 1]
 [0 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [1 1 0 ... 0 1 0]
 [1 0 0 ... 0 1 0]]


### Data exploration

In [36]:
# Exploring the data: mean, median, standard deviation of STAI
overall_mean = np.mean(stai_scores)
overall_std = np.std(stai_scores)
overall_median = np.median(stai_scores)

anxious_mean = np.mean(stai_scores[0:24])
anxious_std = np.std(stai_scores[0:24])
anxious_median = np.median(stai_scores[0:24])

control_mean = np.mean(stai_scores[25:])
control_std = np.std(stai_scores[25:])
control_median = np.median(stai_scores[25:])

In [35]:
# Exploring the data: cut-off = 43, ANXIOUS group if STAI>43
cut_off_stai = np.zeros(len(stai_scores))
for i in range(len(stai_scores)):
    if stai_scores[i] <= 43:
        # cut_off_stai = 1 if the subject is in the healthy control group
        cut_off_stai[i] = 1

healthy_num = sum(cut_off_stai)
healthy_index = np.where(cut_off_stai==1)

In [59]:
# Exploring the data: number of times each subject choosing stimuli A
chose_a_count = []
chose_a_percent = []
for i in range(len(inst_choices)):
    chose_a_count.append(np.array(np.where(inst_choices[i]==1)).size)
    chose_a_percent.append(chose_a_count[-1]/len(inst_choices[i]))

### Simulation


In [160]:
# Value of a chosen stimuli i at time t
# the probability of each stimulus to lead to the aversive noise, 70/30, 80/20, 60/40, 65/35
prob_list = [[0.3,0.7],[0.2,0.8],[0.4,0.6],[0.35,0.65]]

def gen_outcome(a):
    first = np.random.choice([0, 1], size=40, p=a[0])
    second = np.random.choice([0, 1], size=40, p=a[1])
    third = np.random.choice([0, 1], size=40, p=a[2])
    forth = np.random.choice([0, 1], size=40, p=a[3])
    array = np.concatenate((first,second,third,forth))
    return array

def stimuli_value(V0, a, outcome):
    value = np.zeros(len(outcome.T))
    value[0] = V0
    for i in range(2,len(outcome.T)):
        value[i] = value[i-1] + a*(outcome[i-1]-value[i-1])
    return value

In [192]:

np.random.seed(1)
outcome_a = gen_outcome(prob_list)
outcome_b = 1 - outcome_a
value_a = stimuli_value(0.5,0.4,outcome_a)
value_b = stimuli_value(0.5,0.4,outcome_b)